In [ ]:
import pandas as pd
from utilities.evaluation import ModelEvaluation
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

amlcompute_name = 'cpu-cluster'

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os

# Specify parameter sampler

params = {'--C': uniform(0.1, 1),
          '--max_iter': quniform(100, 1500, 100)}

ps = RandomParameterSampling(params)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./src',
              compute_target=aml_compute,
              entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=25,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run_hyperdrive = exp.submit(config=hyperdrive_config)

In [ ]:
RunDetails(run_hyperdrive).show()

In [ ]:
best_run = run_hyperdrive.get_best_run_by_primary_metric()
model = best_run.register_model(model_name='bankmarketing-logreg', model_path='models/logistic_regression.joblib')

model.download(target_dir='models', exist_ok=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.
logistic = joblib.load('models/logistic_regression.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv"
ds = TabularDatasetFactory().from_delimited_files(path)

In [ ]:
from src.train import clean_data

# Use the clean_data function to clean your data.
test_data, test_label = clean_data(ds)
evaluation = ModelEvaluation(observed=test_label, predicted=logistic.predict(test_data))
evaluation.calculate_metrics()
evaluation.print_metrics()
evaluation.confusion_matrix(normalize=False)

## AutoML Pipeline

In [ ]:
experiment_automl = Experiment(workspace=ws, name='udacity-automl')

print(f'Workspace name: {ws.name}',
      f'Azure region: {ws.location}',
      f'Resource group: {ws.resource_group}', sep='\n')

In [ ]:
datastore = ws.get_default_datastore()

factory = TabularDatasetFactory()
path_train_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
path_test_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv'
path_valid_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_valid.csv'

ds_train = factory.from_delimited_files(path_train_data)
ds_test = factory.from_delimited_files(path_test_data)
ds_valid = factory.from_delimited_files(path_valid_data)

In [ ]:
import logging
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

label = 'y'

automl_settings = {
    'enable_early_stopping': True,
    'iteration_timeout_minutes': 5,
    'max_concurrent_iterations': 4,
    'max_cores_per_iteration': -1,
    'primary_metric': 'accuracy',
    'featurization': 'auto',
    'verbosity': logging.INFO
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    debug_log='automl_errors.log',
    compute_target=aml_compute,
    training_data=ds_train,
    validation_data=ds_valid,
    label_column_name=label,
    n_cross_validations=5)

In [2]:
# Submit your automl run

remote_run = experiment_automl.submit(automl_config, show_output=False)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
# Retrieve and save your best automl model.

best_run_aml, fitted_model_aml = remote_run.get_output()
model_name = best_run_aml.properties['model_name']

In [ ]:
import pickle

best_run_aml.download_file('models/model.pkl', 'models/aml-model.pkl')

aml_model = pickle.load(open('models/aml-model.pkl', 'rb'))

In [ ]:
test_data = pd.read_csv(path_test_data)

y_test = test_data.pop('y')

aml_evaluation = ModelEvaluation(observed=y_test, predicted=aml_model.predict(test_data))
aml_evaluation.calculate_metrics()
aml_evaluation.print_metrics()
aml_evaluation.confusion_matrix(normalize=False)

In [ ]:
try:
    aml_compute.delete()
    print('Compute target deleted!')
except ComputeTargetException:
    print('Compute target not found!')
